# Model

In [26]:
from transformers import AutoModel, AutoTokenizer
import torch

model_name = "5CD-AI/Vintern-1B-v3_5"

model = AutoModel.from_pretrained(
    model_name,
    trust_remote_code=True
).cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False, padding='right')

# Dataset

In [8]:
from datasets import load_dataset
dataset = load_dataset('5CD-AI/Vietnamese-Multi-turn-Chat-Alpaca')

In [9]:
def prepare_dataset(sample):
    processed_sample = []
    for turn in sample['conversations']:
        processed_turn = {}
        if turn['from'] == 'human':
            processed_turn['role'] = 'user'
        elif turn['from'] == 'gpt':
            processed_turn['role'] = 'assistant'
        processed_turn['content'] = turn['value']
        processed_sample.append(processed_turn)
    return {'conversations':processed_sample}
    

In [10]:
# Apply dataset preparation
print("Preparing dataset...")
dataset = dataset['train'].map(prepare_dataset, remove_columns=['conversations', 'id'], num_proc=4)
print(f"Processed dataset sample: {dataset[0]['conversations']}")

# Apply chat template (no pixel_values here)
print("Applying chat template...")
dataset = dataset.map(lambda x: {'text': tokenizer.apply_chat_template(x['conversations'], tokenize=False)}, num_proc=4, remove_columns=['conversations'])
print(f"Formatted sample: {dataset[0]['text'][:200]}...")


Preparing dataset...
Processed dataset sample: [{'content': 'Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "Trận đấu là một thất bại nặng nề mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần."', 'role': 'user'}, {'content': 'Nhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề.', 'role': 'assistant'}, {'content': 'Bạn có thể đề xuất một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của họ trong trận đấu tiếp theo không?', 'role': 'user'}, {'content': 'Chắc chắn, đây là một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của mình trong trận đấu tiếp theo: 1. Phân tích trận đấu trước bằng cách xem lại cảnh quay trận đấu để xác định điểm yếu và các lĩnh vực cần cải thiện. 2. Tăng cường độ và sự tập trung của các buổi tập để đảm bảo mọi cầu thủ đều được chuẩn bị đầy đủ về thể chất và tinh thần. 3. Luyện tập những kỹ năng cụ thể cần cải thiện, chẳng hạn như sút bóng hoặc chuyền bóng chính xác. 4. Phát triển các chiến lược và chiến thu

Map (num_proc=4):   0%|          | 0/12697 [00:00<?, ? examples/s]

Formatted sample: <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "Trận đấu là một thất ...


In [16]:
def tokenize_dataset(sample):
    return tokenizer(sample['text'], return_tensors=None)

dataset = dataset.map(tokenize_dataset, remove_columns=['text'])

Map:   0%|          | 0/12697 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2035 > 1700). Running this sequence through the model will result in indexing errors


In [21]:
from transformers import DataCollatorForLanguageModeling

# Custom data collator to add dummy pixel_values
class CustomDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    def __call__(self, examples):
        texts = [ex['input_ids'] for ex in examples]
        encodings = self.tokenizer.pad(
            {'input_ids':texts},
            padding=True,
            max_length=512,
            return_tensors='pt'
        )
        batch = {
            'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask'],
        }

        batch_size = len(examples)
        batch['pixel_values'] = torch.zeros((batch_size, 3, 448, 448), dtype=torch.float16)
        batch['image_flags'] = torch.tensor((batch_size,0), dtype=torch.int)
        return batch

In [27]:
from transformers import TrainingArguments, Trainer
import os
from torch.utils.data import DataLoader

# Disable W&B to avoid hangs
os.environ["WANDB_DISABLED"] = "true"

# Define training arguments
training_args = TrainingArguments(
    output_dir="./qwen2_finetuned",
    per_device_train_batch_size=1,  # Minimize memory usage
    gradient_accumulation_steps=8,  # Effective batch size = 8
    learning_rate=2e-5,
    num_train_epochs=1,  # Start with 1 epoch
    logging_steps=1,  # Log every step
    logging_dir="./logs",
    fp16=True,  # Mixed precision
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none",  # Disable all logging integrations
)

class CustomTrainer(Trainer):
    def get_train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.args.per_device_train_batch_size,
            collate_fn=self.data_collator,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
        )
    def get_eval_dataloader(self, eval_dataset=None):
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        return DataLoader(
            eval_dataset,
            batch_size=self.args.per_device_eval_batch_size,
            collate_fn=self.data_collator,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
            )

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=CustomDataCollator(tokenizer=tokenizer),
)

# Start fine-tuning
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/parallel_apply.py", line 96, in _worker
    output = module(*input, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/.cache/huggingface/modules/transformers_modules/5CD-AI/Vintern-1B-v3_5/115975aca0407d3c54bb0beb8a1da54d31de3f20/modeling_internvl_chat.py", line 107, in forward
    vit_embeds = self.extract_feature(pixel_values)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/.cache/huggingface/modules/transformers

In [21]:
help(model.forward)

Help on method forward in module transformers_modules.5CD-AI.Vintern-1B-v3_5.115975aca0407d3c54bb0beb8a1da54d31de3f20.modeling_internvl_chat:

forward(pixel_values: torch.FloatTensor, input_ids: torch.LongTensor = None, attention_mask: Optional[torch.Tensor] = None, position_ids: Optional[torch.LongTensor] = None, image_flags: Optional[torch.LongTensor] = None, past_key_values: Optional[List[torch.FloatTensor]] = None, labels: Optional[torch.LongTensor] = None, use_cache: Optional[bool] = None, output_attentions: Optional[bool] = None, output_hidden_states: Optional[bool] = None, return_dict: Optional[bool] = None) -> Union[Tuple, transformers.modeling_outputs.CausalLMOutputWithPast] method of transformers_modules.5CD-AI.Vintern-1B-v3_5.115975aca0407d3c54bb0beb8a1da54d31de3f20.modeling_internvl_chat.InternVLChatModel instance
    Define the computation performed at every call.
    
    Should be overridden by all subclasses.
    
    .. note::
        Although the recipe for forward pa